**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [3]:
!pip install sk-video

    100% |████████████████████████████████| 2.3MB 10.6MB/s 


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?


In the training phase (train=True), the function uses the currently learned policy with probability $1-\epsilon$ and randomly picks an action with probability $\epsilon$. This is one way to balance exploration and exploitation which is called $\epsilon$-greedy. (other approaches (UCB) favor less explored actions and states through the higher uncertainies on their estimated discounted reward).

The $\epsilon$ is necessary to perform exploration and form accurate estimates of the Q-function of the current policy in order to improve properly.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=15 # set small when debugging
epochs_test=15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.


board is a 2D array that stores food values on the NxN grid (+0.5 for cheese and -1 for poisonous cells). And position is a 2D array that stores the current position of the rat (+1 for the current positon, -1 elsewhere)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            if reward > 0 :
                win += reward
            else:
                lose -= reward          
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 0.5/4.0. Average score (-3.5)
Win/lose count 2.0/2.0. Average score (-1.75)
Win/lose count 2.5/2.0. Average score (-1.0)
Win/lose count 3.0/2.0. Average score (-0.5)
Win/lose count 1.5/1.0. Average score (-0.3)
Win/lose count 1.0/3.0. Average score (-0.5833333333333334)
Win/lose count 1.5/1.0. Average score (-0.42857142857142855)
Win/lose count 1.0/4.0. Average score (-0.75)
Win/lose count 1.5/0.0. Average score (-0.5)
Win/lose count 2.0/3.0. Average score (-0.55)
Win/lose count 1.0/2.0. Average score (-0.5909090909090909)
Win/lose count 2.0/3.0. Average score (-0.625)
Win/lose count 2.0/2.0. Average score (-0.5769230769230769)
Win/lose count 2.5/1.0. Average score (-0.42857142857142855)
Win/lose count 1.5/1.0. Average score (-0.36666666666666664)
Final score: -0.36666666666666664


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**Answer**
***
We have: 
\begin{align*} 
Q^\pi (s,a) &= E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_t, a_t)|s_0 = s, a_0 = a] \\
 &= E_{p^\pi} [r(s_0,a_0) + \gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a] 
\end{align*}

The first term is $E_{p^\pi} [r(s_0,a_0)|s_0 = s, a_0 = a] = r(s, a)$

The second term can be simplified as:

\begin{align*}
\gamma E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a] &= \gamma E_{(s',a')\sim p^\pi(.|s, a)}[E_{p^\pi} [\sum_{t=1}^\infty \gamma^t r(s_{t+1}, a_{t+1}) | s_0 = s, s_1=s', a_0=a, a_1=a']]\\
  \end{align*}
  And using the Markov property, this is equal to:
 \begin{align*}
 \gamma E_{(s',a')\sim p^\pi(.|s, a)}[E_{p^\pi} [\sum_{t=0}^\infty \gamma^{t-1} r(s_t, a_t) |  s_1=s',  a_1=a']]&=
 \gamma E_{(s',a')\sim p^\pi(.|s, a)}[Q^\pi(s',a')]
 \end{align*}
 Which concludes the first question.
The previous calculations also show that:
\begin{align*}
Q^{\pi}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma v^{\pi}(s')]
\end{align*}
An using the policy improvement theorem, we know that an optimal policy $v^*$ satisfies $v^*(s) = \mbox{max}_{a}Q^* (s,a)$. This leads to:

\begin{align*}
Q^*(s,a) &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)  +\gamma v^*(s')] \\
&=E_{s'\sim \pi^*(.|s,a)}[r(s,a)  +\gamma \max_{a'}Q^*(s',a')]
\end{align*}

We minimize $\mathcal{L}$ in hopes of finding $\theta$ such that $\mathcal{L}(\theta)=0$. The closer this metric is to 0, the closer we are to finding an optimal policy.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None, ...]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        shape = (1, 5, 5, self.n_state)
        for i in range(self.batch_size):
            ######## FILL IN
            s, n_s, a, r, game_over_  = self.memory.random_access()
            q_s_a = self.model.predict(n_s.reshape(shape))
            target_q[i] = q_s_a
            
            if game_over_:
                ######## FILL IN
                input_states[i] = s
                target_q[i,a] = r
            else:
                ######## FILL IN
                q_s_a = self.model.predict(n_s.reshape(shape))
                input_states[i] = s
                target_q[i,a]   = r  + self.discount*np.max(q_s_a)
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(keras.layers.Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(120,activation ='relu'))
        model.add(Dense(40,activation ='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [19]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 000/015 | Loss 0.0190 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/015 | Loss 0.0048 | Win/lose count 4.5/7.0 (-2.5)
Epoch 002/015 | Loss 0.0650 | Win/lose count 3.5/8.0 (-4.5)
Epoch 003/015 | Loss 0.0133 | Win/lose count 3.5/7.0 (-3.5)
Epoch 004/015 | Loss 0.0148 | Win/lose count 5.0/5.0 (0.0)
Epoch 005/015 | Loss 0.0178 | Win/lose count 5.5/1.0 (4.5)
Epoch 006/015 | Loss 0.0191 | Win/lose count 9.0/6.0 (3.0)
Epoch 007/015 | Loss 0.0102 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/015 | Loss 0.0102 | Win/lose count 4.0/2.0 (2.0)
Epoch 009/015 | Loss 0.0077 | Win/lose count 6.5/4.0 (2.5)
Epoch 010/015 | Loss 0.0099 | Win/lose count 10.0/8.0 (2.0)
Epoch 011/015 | Loss 0.0101 | Win/lose count 10.5/10.0 (0.5)
Epoch 012/015 | Loss 0.0089 | Win/lose count 7.0/8.0 (-1.0)
Epoch 013/015 | Loss 0.0087 | Win/lose count 4.0/3.0 (1.0)
Epoch 014/015 | Loss 0.0153 | Win/lose count 5

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(filters=32,kernel_size=(3,3),input_shape=(5,5,self.n_state),strides=(1,1),padding="same",activation="relu"))
        model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation="relu"))
        model.add(keras.layers.Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [21]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/015 | Loss 0.0051 | Win/lose count 1.5/4.0 (-2.5)
Epoch 001/015 | Loss 0.0058 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/015 | Loss 0.0077 | Win/lose count 3.0/2.0 (1.0)
Epoch 003/015 | Loss 0.0726 | Win/lose count 7.0/13.0 (-6.0)
Epoch 004/015 | Loss 0.0080 | Win/lose count 2.0/4.0 (-2.0)
Epoch 005/015 | Loss 0.0103 | Win/lose count 8.0/5.0 (3.0)
Epoch 006/015 | Loss 0.0043 | Win/lose count 6.5/3.0 (3.5)
Epoch 007/015 | Loss 0.0637 | Win/lose count 10.5/4.0 (6.5)
Epoch 008/015 | Loss 0.0055 | Win/lose count 13.0/3.0 (10.0)
Epoch 009/015 | Loss 0.0050 | Win/lose count 19.0/2.0 (17.0)
Epoch 010/015 | Loss 0.0689 | Win/lose count 10.5/4.0 (6.5)
Epoch 011/015 | Loss 0.0104 | Win/lose count 16.0/8.0 (8.0)
Epoch 012/015 | Loss 0.0096 | Win/lose count 17.5/2.0 (15.5)
Epoch 013/015 | Loss 0.0029 | Win/lose count 17.0/7.0 (10.0)
Epoch 014/015 | Loss 0.0095 | Win/lose count 7.5/6.0 (1.5)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.5/3.0. Average score (-0.5)
Win/lose count 6.5/5.0. Average score (0.5)
Win/lose count 3.0/3.0. Average score (0.3333333333333333)
Win/lose count 2.5/3.0. Average score (0.125)
Win/lose count 2.5/4.0. Average score (-0.2)
Win/lose count 4.5/1.0. Average score (0.4166666666666667)
Win/lose count 7.5/6.0. Average score (0.5714285714285714)
Win/lose count 1.0/4.0. Average score (0.125)
Win/lose count 4.0/2.0. Average score (0.3333333333333333)
Win/lose count 9.0/5.0. Average score (0.7)
Win/lose count 8.5/4.0. Average score (1.0454545454545454)
Win/lose count 13.5/6.0. Average score (1.5833333333333333)
Win/lose count 9.5/5.0. Average score (1.8076923076923077)
Win/lose count 5.5/8.0. Average score (1.5)
Win/lose count 3.0/7.0. Average score (1.1333333333333333)
Final score: 1.1333333333333333
Test of the FC
Win/lose count 3.5/3.0. Average score (0.5)
Win/lose count 2.5/3.0. Average score (0.0)
Win/lose count 1.5/3.0. Average score (-0.5)
Win/lose count 4.

In [23]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))


I tried to track the performance of each model while changing the temperature parameter. I noticed that CNN does better than FCN for almost all temperatures, however, it requires more time to train. This improvement from FCN to CNN is more noticeable with higher temperatures.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    decay = 0.85
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*decay)  # Epsilon greedy policy
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
   
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        
    def act(self, action , train = True):
        state, reward, game_over = super(EnvironmentExploring,self).act(action)
        # Exploration
        if train:
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(         self.grid_size, self.grid_size,1),
                                self.position.reshape(      self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over
    
    def reset(self):
        _ = super(EnvironmentExploring,self).reset()
        self.malus_position = np.zeros((self.grid_size,self.grid_size))    
        self.malus_position[self.x, self.y] = 0.1
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(         self.grid_size, self.grid_size,1),
                                self.position.reshape(      self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [30]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.6, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/015 | Loss 0.0153 | Win/lose count 7.0/21.900000000000052 (-14.900000000000052)
Epoch 001/015 | Loss 0.0202 | Win/lose count 6.0/25.7000000000001 (-19.7000000000001)
Epoch 002/015 | Loss 0.0120 | Win/lose count 5.5/24.20000000000007 (-18.70000000000007)
Epoch 003/015 | Loss 0.0250 | Win/lose count 7.5/30.60000000000012 (-23.10000000000012)
Epoch 004/015 | Loss 0.0194 | Win/lose count 11.0/18.400000000000002 (-7.400000000000002)
Epoch 005/015 | Loss 0.0112 | Win/lose count 7.5/23.200000000000053 (-15.700000000000053)
Epoch 006/015 | Loss 0.0311 | Win/lose count 16.5/17.19999999999998 (-0.6999999999999815)
Epoch 007/015 | Loss 0.0212 | Win/lose count 8.5/20.30000000000002 (-11.800000000000018)
Epoch 008/015 | Loss 0.0142 | Win/lose count 10.0/17.79999999999999 (-7.79999999999999)
Epoch 009/015 | Loss 0.0099 | Win/lose count 17.5/17.19999999999998 (0.3000000000000185)
Epoch 010/015 | Loss 0.0151 | Win/lose count 16.5/15.09999999999997 (1.4000000000000306)
Epoch 011/015 | Loss 0.

In [31]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 21.5/20.20000000000002. Average score (1.2999999999999794)
Win/lose count 18.5/15.999999999999977. Average score (1.9000000000000012)
Win/lose count 16.5/14.29999999999997. Average score (2.000000000000011)
Win/lose count 18.5/15.999999999999972. Average score (2.125000000000016)
Win/lose count 22.5/13.799999999999978. Average score (3.4400000000000177)
Win/lose count 23.0/23.700000000000053. Average score (2.7500000000000058)
Win/lose count 21.0/22.000000000000043. Average score (2.214285714285713)
Win/lose count 23.5/16.499999999999982. Average score (2.8125000000000013)
Win/lose count 19.0/17.19999999999999. Average score (2.700000000000003)
Win/lose count 18.5/18.699999999999992. Average score (2.4100000000000033)
Win/lose count 17.0/18.100000000000005. Average score (2.090909090909094)
Win/lose count 24.0/14.699999999999969. Average score (2.6916666666666718)
Win/lose count 29.0/14.099999999999977. Average score (3.6307692307692374)
Win/lose count 25.0/14.2999999999

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***